In [25]:
import pandas as pd
import requests

def metrojson(SID, EID):

    SID = str(SID)
    EID = str(EID)
    Sopt = '1'   # 최단거리(1) 또는 최소환승(2) 설정
    apiKey = "VMlJA4GzIlpVQkHARsMsz6uOKDrPKxiHBCfYXoz6ufo"

    url = f'https://api.odsay.com/v1/api/subwayPath?lang=0&CID=1000&SID={SID}&EID={EID}&Sopt={Sopt}&apiKey={apiKey}'
    
    res = requests.get(url)
    data = res.json()

    # DataFrame으로 만들 데이터 추출
    globalStartName = data["result"]["globalStartName"]
    globalEndName = data["result"]["globalEndName"]
    globalTravelTime = data["result"]["globalTravelTime"]
    globalDistance = data["result"]["globalDistance"]
    globalStationCount = data["result"]["globalStationCount"]
    fare = data["result"]["fare"]
    cashFare = data["result"]["cashFare"]
    driveInfoSet = data["result"]["driveInfoSet"]
    stationSet = data["result"]["stationSet"]

    # 'driveInfo'와 'stations' 리스트를 DataFrame으로 변환
    drive_info_df = pd.DataFrame(driveInfoSet["driveInfo"])
    stations_df = pd.DataFrame(stationSet["stations"])
    
    # 결과 데이터를 하나의 DataFrame으로 합치기
    result_df = pd.DataFrame({
        "globalStartName": [globalStartName],
        "globalEndName": [globalEndName],
        "globalTravelTime": [globalTravelTime],
        "globalDistance": [globalDistance],
        "globalStationCount": [globalStationCount],
        "fare": [fare],
        "cashFare": [cashFare]
    })

    def get_direction(row):
        if row["startName"] == "시청" and row["endName"] == "충정로":
            return "외선"
        elif (200 <= row["startID"] <= 299) and (200 <= row["endSID"] <= 299):
            return "내선" if row["startID"] < row["endSID"] else "외선"
        else:
            return "하선" if row["startID"] > row["endSID"] else "상선"
    stations_df["direction"] = stations_df.apply(get_direction, axis=1)
    
    stations_df["time_difference"] = stations_df["travelTime"].diff()
    stations_df.at[stations_df.index[0], "time_difference"] = stations_df.at[stations_df.index[0], "travelTime"]

    total_time_difference_sum = stations_df["time_difference"].sum()
    stations_df["time_difference_percentage"] = (stations_df["time_difference"] / total_time_difference_sum) * 100

    # 결과 DataFrame 출력
    return result_df, drive_info_df, stations_df

In [26]:
result_df, drive_info_df, df_stations = metrojson('426', '210')

In [27]:
result_df

,globalStartName,globalEndName,globalTravelTime,globalDistance,globalStationCount,fare,cashFare
0,서울역,뚝섬,17,8.3,9,1250,1350


In [28]:
drive_info_df

,laneID,laneName,startName,stationCount,wayCode,wayName
0,4,4호선,서울역,4,1,진접
1,2,2호선,동대문역사문화공원,5,2,내선순환


In [29]:
df_stations

,startID,startName,endSID,endName,travelTime,direction,time_difference,time_difference_percentage
0,426,서울역,425,회현,2,하선,2.0,11.764706
1,425,회현,424,명동,4,하선,2.0,11.764706
2,424,명동,423,충무로,5,하선,1.0,5.882353
3,423,충무로,422,동대문역사문화공원,7,하선,2.0,11.764706
4,205,동대문역사문화공원,206,신당,10,내선,3.0,17.647059
5,206,신당,207,상왕십리,12,내선,2.0,11.764706
6,207,상왕십리,208,왕십리,13,내선,1.0,5.882353
7,208,왕십리,209,한양대,15,내선,2.0,11.764706
8,209,한양대,210,뚝섬,17,내선,2.0,11.764706


In [10]:
from API import Odsay

stations = Odsay.station_list(df_stations) # station list 구하는 함수
counts = [drive_info_df['stationCount'][i] for i in range(len(drive_info_df))] # 역 별 이동 역 개수
lane = [drive_info_df['laneName'][i] for i in range(len(drive_info_df))] # 호선 정보

In [11]:
stations

['서울역', '회현', '명동', '충무로', '동대문역사문화공원', '신당', '상왕십리', '왕십리', '한양대']

In [12]:
counts

[4, 5]

In [13]:
lane

['4호선', '2호선']

In [15]:
split_stations = []
split_congestion = []

congestion_ls = [12, 34, 1, 99, 12, 34, 34, 24, 11]

for count in counts:
    split_stations.append(stations[:count])
    split_congestion.append(congestion_ls[:count])
    del stations[:count]

In [16]:
split_congestion

[[12, 34, 1, 99], [12, 34, 1, 99, 12]]

In [17]:
split_stations

[['서울역', '회현', '명동', '충무로'], ['동대문역사문화공원', '신당', '상왕십리', '왕십리', '한양대']]

In [30]:
# 시간 구하기: 윤수님

travle_time_ls = list(df_stations['travelTime'])
percent_ls = list(df_stations['time_difference_percentage'])

In [31]:
travle_time_ls

[2, 4, 5, 7, 10, 12, 13, 15, 17]

In [32]:
percent_ls

[11.76470588235294,
 11.76470588235294,
 5.88235294117647,
 11.76470588235294,
 17.647058823529413,
 11.76470588235294,
 5.88235294117647,
 11.76470588235294,
 11.76470588235294]